In [30]:
async def initialize_browser():
    from playwright.async_api import async_playwright

    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context()
    page = await context.new_page()
    return playwright, browser, context, page

In [31]:
import pandas as pd
from bs4 import BeautifulSoup

playwright, browser, context, page = await initialize_browser()

await page.goto('https://steamdb.info/sales/')

page_num = 1

while await page.query_selector(f"button.dt-paging-button.next") is not None:
    print(f"Page {page_num}")
    headers = []
    contents = []
    html_content = await page.content()
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find(id='DataTables_Table_0')
    if table:
        headers = [header.text for header in table.find_all('th')][2:]
        print(headers)
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            if columns:
                content = [column.text for column in columns][2:]
                if content[0].find('\n\n') != -1:
                    content[0] = content[0].split('\n\n')[0].replace('\n', '')
                contents.append(content)
        print(contents)
    if headers and contents:
        df = pd.DataFrame(contents, columns=headers)
        df.to_csv(f"raw/steamdb_sales_page_{page_num}.csv", index=False)
    page_num += 1
    if await page.query_selector("button.dt-paging-button.disabled.next") is not None:
        break
    await page.wait_for_selector("button.dt-paging-button.next")
    await page.click("button.dt-paging-button.next")
    await page.wait_for_load_state("domcontentloaded")

await browser.close()
await playwright.stop()

Page 1
['Name', '%', ' Price', ' Rating', ' Release', ' Ends', ' Started']
[['Sea Power : Naval Combat in the Missile Age', '-10%', 'R$ 121,49', '81.22%', 'Nov 2024', '', ''], ["Minishoot' Adventures", '-25%', 'R$ 29,99', '93.93%', 'Apr 2024', '', ''], ['UNDERDOGS', '-42%', 'R$ 51,61', '92.48%', 'Jan 2024', '', ''], ['STORY OF SEASONS: Friends of Mineral Town', '-75%', 'R$ 23,49', '90.72%', 'Jul 2020', '', ''], ['Red Orchestra 2: Heroes of Stalingrad with Rising Storm', '-90%', 'R$ 4,99', '90.61%', 'Sep 2011', '', ''], ['Wall World', '-70%', 'R$ 5,09', '89.08%', 'Apr 2023', '', ''], ['Anger Foot', '-30%', 'R$ 51,79', '88.88%', 'Jul 2024', '', ''], ['Rune Factory 3 Special', '-60%', 'R$ 55,99', '88.66%', 'Sep 2023', '', ''], ['Maniac', '-35%', 'R$ 11,04', '87.22%', 'Mar 2024', '', ''], ['Coffee Caravan', '-25%', 'R$ 28,86', '86.59%', 'May 2024', '', ''], ['Rising Storm 2: Vietnam', '-95%', 'R$ 2,29', '85.30%', 'May 2017', '', ''], ['Ratopia', '-25%', 'R$ 44,99', '84.62%', 'Nov 2023', ''